In [1]:
import arcpy
import requests
from arcpy.sa import Con
from io import BytesIO
from py7zr import SevenZipFile

In [ ]:
#First step is calling in our raster data
#We will be using the request tool to download the needed zip files
#First is our needed temp data
url = "https://gmed.auckland.ac.nz/data/btemp_k.7z"
response = requests.get(url)

if response.status_code == 200:
    zip_data = BytesIO(response.content)
    with ZipFile(zip_data, 'r') as z:
        z.extractall()

    print("Extraction complete.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

In [ ]:
#Then our depth data
url = "https://gmed.auckland.ac.nz/data/depth.7z"

response = requests.get(url)

if response.status_code == 200:
    zip_data = BytesIO(response.content)
    with SevenZipFile(zip_data, mode='r') as z:
        z.extractall()

    print("Extraction complete.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

In [ ]:
#And then our pH data
url = "https://gmed.auckland.ac.nz/data/ph.7z"

response = requests.get(url)

if response.status_code == 200:
    zip_data = BytesIO(response.content)
    with SevenZipFile(zip_data, mode='r') as z:
        z.extractall()

    print("Extraction complete.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

Due to diffciculties in getting the API of the Global Fishing Watch to work, the global fishing data will need to be manually loaded in.

In [ ]:
#First step is to take our raster layers and adjust their values using fizzy functions. 
#First we will do our ocean temp raster.
input_raster = "kg_b_temp.asc"
output_raster = "fuzzy_temp"

# Here, we will perform the conditional calculation using arcpy.sa.Con
fuzzy_temp = Con((arcpy.Raster(input_raster) >= 22.778) & (arcpy.Raster(input_raster) <= 28.889),
                    1,
                    Con(arcpy.Raster(input_raster) < 22,
                        arcpy.Raster(input_raster) / 28,
                        Con(arcpy.Raster(input_raster) > 28,
                            (28 - arcpy.Raster(input_raster)) / 28,
                            0)))

print("Calculation completed successfully.")

In [ ]:
#Next we do the same for ocean pH
input_raster = "bo_ph.asc"
output_raster = "fuzzy_ph"

fuzzy_ph = Con(arcpy.Raster(input_raster) >= 8.1,
                    1,
                    Con(arcpy.Raster(input_raster) < 8.1,
                        arcpy.Raster(input_raster) / 8.61972,
                        0))

print("Calculation completed successfully.")

In [ ]:
#Then we create a fuzzy function for ocean depth
input_raster = "gb_depth.asc"
output_raster = "fuzzy_depth"

fuzzy_depth = Con(arcpy.Raster(input_raster) >= -70,
                    1,
                    Con(arcpy.Raster(input_raster) < -70,
                        (-10293.7 - arcpy.Raster(input_raster)) / -10293.7,
                        0))

print("Calculation completed successfully.")

In [2]:
#Next step is our fishing activity raster
original_raster_path = "public_global_fishing.tif"
#Since this raster only includes cells denoting fishing activity, meaning areas where no fishing was observed are not included, we should fill in the gaps
#To do so, first we create a raster object
public_fishing_total = arcpy.Raster(original_raster_path)

#Then we can create a binary raster where NoData is 1 and data is 0
public_fishing_total = arcpy.sa.IsNull(public_fishing_total)

In [ ]:
#We can then merge the binary raster back to the original fishing raster as a mosaic.
input_raster1 = "public_fishing_total"
input_raster2 = "public_global_fishing.tif"


output_mosaic = "public_fishing_whole"
input_rasters = [input_raster1, input_raster2]

#After establishing our rasters, we can use ArcGIS Pro's MosaicToNewRaster to merge the rasters
arcpy.management.MosaicToNewRaster(input_rasters, arcpy.env.workspace, output_mosaic,
                                   pixel_type="32_BIT_FLOAT", number_of_bands=1)

print("Mosaic completed successfully.")

In [ ]:
#With a comprehensive ocean fishing layer now ready, we can create our final fuzzy function
input_raster = "public_fishing_whole"
output_raster = "fuzzy_fish"


fuzzy_fish = Con(arcpy.Raster(input_raster) == 1,
                    1,
                    Con(arcpy.Raster(input_raster) > 0,
                        (426638 - arcpy.Raster(input_raster)) / 426638,
                        0))

print("Calculation completed successfully.")

In [9]:
#Now that our layers are finished being prepared, we can create the weighted overlay
#First, we specify the paths to your raster files
fuzzy_pH_path = "fuzzy_pH"
fuzzy_depth_path = "fuzzy_depth"
fuzzy_temp_path = "fuzzy_temp"
fuzzy_fishing_path = "fuzzy_fish"

#We then turn the layers into raster objects
fuzzy_layer1 = arcpy.Raster(fuzzy_pH_path)
fuzzy_layer2 = arcpy.Raster(fuzzy_depth_path)
fuzzy_layer3 = arcpy.Raster(fuzzy_temp_path)
fuzzy_layer4 = arcpy.Raster(fuzzy_fishing_path)

#Then we can create weights
weight1 = 0.3
weight2 = 0.3
weight3 = 0.3
weight4 = 0.1

#Finally, we can create the weighted overlay by pluggin everything above into the following equation
weighted_overlay = (fuzzy_layer1 * weight1) + (fuzzy_layer2 * weight2) + (fuzzy_layer3 * weight3) + (fuzzy_layer4 * weight4)


In [10]:
#Now that we have the overlay, we can reclass the data into five classes
reclass_overlay = arcpy.sa.Reclassify(
    in_raster="weighted_overlay",
    reclass_field="VALUE",
    remap="0 0.587445 1;0.587445 0.647369 2;0.647369 0.737256 3;0.737256 0.880152 4;0.880152 1 5",
    missing_values="DATA"
)

In [ ]:
#In order to determine how many coral reefs are in each class, we will need to spatially join that data to the reef polgons
#First, we must convert the rasters to vectors
coral_reefs = "WCMC008_CoralReef2021_Py_v4_1"
ocean_raster = "reclass_overlay"

arcpy.conversion.RasterToPolygon(
    in_raster=ocean_raster,
    out_polygon_features="ocean_raster_polygons",
    simplify="NO_SIMPLIFY"
)

#Then we can perform a spatial join to append the values of the ocean raster polygons to the coral reef polygons
arcpy.analysis.SpatialJoin(
    target_features=coral_reefs,
    join_features="ocean_raster_polygons",
    out_feature_class="coral_reefs_with_raster_values",
    join_type="KEEP_COMMON"
)
#With the overlay class data now joined to polygons, we can calculate how many reefs are in each class
#Here, we can use the Summary Statistics tool to calculate the count of coral reefs
arcpy.analysis.Statistics(
    in_table="coral_reefs_with_raster_values",
    out_table="coral_reefs_statistics",
    statistics_fields=[("Shape_Length", "COUNT")],
    case_field="gridcode"
)
# Finally, we can print the results
with arcpy.da.SearchCursor("coral_reefs_statistics", ["gridcode", "COUNT_Shape_Length"]) as cursor:
    for row in cursor:
        raster_value = row[0]
        count = row[1]
        print(f"Raster Value {raster_value}: {count} coral reefs")


#We can also clean up intermediate data through the following code
arcpy.management.Delete("ocean_raster_polygons")
arcpy.management.Delete("coral_reefs_with_raster_values")
arcpy.management.Delete("coral_reefs_statistics")


# Verification

In [ ]:
#We have our base overlay, but now we have to have to verify the results
#We will do so using a sensitivity analysis
#We first define the weights of our first verification
weight1 = 0.25
weight2 = 0.25
weight3 = 0.25
weight4 = 0.25


#Then we get the sum the weighted layers
weighted_overlay_verification1 = (fuzzy_layer1 * weight1) + (fuzzy_layer2 * weight2) + (fuzzy_layer3 * weight3) + (fuzzy_layer4 * weight4)

#Then we save the result
output_path = "weighted_overlay_verification1"
weighted_overlay_verification1.save(output_path)

#Then we reclassify
verify_reclass_1 = arcpy.sa.Reclassify(
    in_raster="weighted_overlay_verification1",
    reclass_field="VALUE",
    remap="0 0.656203 1;0.656203 0.706140 2;0.706140 0.781045 3;0.781045 0.900126 4;0.900126 1 5",
    missing_values="DATA"
)

ocean_raster = "verify_reclass_1"

# We then convert the raster to polygons
arcpy.conversion.RasterToPolygon(
    in_raster=ocean_raster,
    out_polygon_features="ocean_raster_polygons",
    simplify="NO_SIMPLIFY"
)

#We perform a spatial join
arcpy.analysis.SpatialJoin(
    target_features=coral_reefs,
    join_features="ocean_raster_polygons",
    out_feature_class="coral_reefs_with_raster_values",
    join_type="KEEP_COMMON"
)
# We create summary statistics
arcpy.analysis.Statistics(
    in_table="coral_reefs_with_raster_values",
    out_table="coral_reefs_statistics",
    statistics_fields=[("Shape_Length", "COUNT")],
    case_field="gridcode"  # Replace with the actual field name from the spatial join that contains raster values
)
# And we print the results
with arcpy.da.SearchCursor("coral_reefs_statistics", ["gridcode", "COUNT_Shape_Length"]) as cursor:
    for row in cursor:
        raster_value = row[0]
        count = row[1]
        print(f"Raster Value {raster_value}: {count} coral reefs")


# Clean up intermediate data if needed
arcpy.management.Delete("ocean_raster_polygons")
arcpy.management.Delete("coral_reefs_with_raster_values")
arcpy.management.Delete("coral_reefs_statistics")


In [ ]:
#We define weights of our second verification
weight1 = 0.40
weight2 = 0.20
weight3 = 0.20
weight4 = 0.20


#We create a sum of the weighted layers
weighted_overlay_verification2 = (fuzzy_layer1 * weight1) + (fuzzy_layer2 * weight2) + (fuzzy_layer3 * weight3) + (fuzzy_layer4 * weight4)

# We save the result
output_path = "weighted_overlay_verification2"
weighted_overlay_verification2.save(output_path)


# We reclassify
verify_reclass_2 = arcpy.sa.Reclassify(
    in_raster="weighted_overlay_verification2",
    reclass_field="VALUE",
    remap="0 0.723426 1;0.723426 0.763375 2;0.763375 0.823300 3;0.823300 0.917028 4;0.917028 1 5",
    missing_values="DATA"
)

ocean_raster = "verify_reclass_2"

# We convert the raster to polygons
arcpy.conversion.RasterToPolygon(
    in_raster=ocean_raster,
    out_polygon_features="ocean_raster_polygons",
    simplify="NO_SIMPLIFY"
)

# We do a spatial join
arcpy.analysis.SpatialJoin(
    target_features=coral_reefs,
    join_features="ocean_raster_polygons",
    out_feature_class="coral_reefs_with_raster_values",
    join_type="KEEP_COMMON"
)

#We get spatial statistics
arcpy.analysis.Statistics(
    in_table="coral_reefs_with_raster_values",
    out_table="coral_reefs_statistics",
    statistics_fields=[("Shape_Length", "COUNT")],
    case_field="gridcode"  # Replace with the actual field name from the spatial join that contains raster values
)
# We print the results
with arcpy.da.SearchCursor("coral_reefs_statistics", ["gridcode", "COUNT_Shape_Length"]) as cursor:
    for row in cursor:
        raster_value = row[0]
        count = row[1]
        print(f"Raster Value {raster_value}: {count} coral reefs")


# Clean up intermediate data
arcpy.management.Delete("ocean_raster_polygons")
arcpy.management.Delete("coral_reefs_with_raster_values")
arcpy.management.Delete("coral_reefs_statistics")


In [ ]:
# For our third verification, we once again define weights
weight1 = 0.20
weight2 = 0.20
weight3 = 0.40
weight4 = 0.20


# We sum the weighted layers
weighted_overlay_verification3 = (fuzzy_layer1 * weight1) + (fuzzy_layer2 * weight2) + (fuzzy_layer3 * weight3) + (fuzzy_layer4 * weight4)

# We save the result
output_path = "weighted_overlay_verification3"
weighted_overlay_verification3.save(output_path)

# We reclassify
verify_reclass_3 = arcpy.sa.Reclassify(
    in_raster="weighted_overlay_verification3",
    reclass_field="VALUE",
    remap="0 0.535388 1;0.535388 0.581163 2;0.581163 0.677289 3;0.677289 0.858099 4;0.858099 1 5",
    missing_values="DATA"
)

ocean_raster = "verify_reclass_3"

# We convert the raster to polygons
arcpy.conversion.RasterToPolygon(
    in_raster=ocean_raster,
    out_polygon_features="ocean_raster_polygons",
    simplify="NO_SIMPLIFY"
)

# We do a spatial join
arcpy.analysis.SpatialJoin(
    target_features=coral_reefs,
    join_features="ocean_raster_polygons",
    out_feature_class="coral_reefs_with_raster_values",
    join_type="KEEP_COMMON"
)
# We get summary statistics
arcpy.analysis.Statistics(
    in_table="coral_reefs_with_raster_values",
    out_table="coral_reefs_statistics",
    statistics_fields=[("Shape_Length", "COUNT")],
    case_field="gridcode"  # Replace with the actual field name from the spatial join that contains raster values
)
# We print the results
with arcpy.da.SearchCursor("coral_reefs_statistics", ["gridcode", "COUNT_Shape_Length"]) as cursor:
    for row in cursor:
        raster_value = row[0]
        count = row[1]
        print(f"Raster Value {raster_value}: {count} coral reefs")


# Clean up intermediate data
arcpy.management.Delete("ocean_raster_polygons")
arcpy.management.Delete("coral_reefs_with_raster_values")
arcpy.management.Delete("coral_reefs_statistics")
